In [1]:
import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf 

from bs4 import BeautifulSoup

import requests

from pyspark.sql import SparkSession

import pyspark

from pyspark.context import SparkContext

import pyspark.sql.functions as fc

import pymysql

from pyspark.sql import Row

from pyspark.sql.types import *

import json

import numpy as np

In [2]:
sc = SparkContext.getOrCreate()

spark = SparkSession(sc)

In [3]:
stockspark = spark.read.json('stockdata.json')

companyspark = spark.read.json('companyinfo.json')

In [4]:
companyspark.show(10)

+--------------------+--------------------+------+
|                Name|              Sector|Symbol|
+--------------------+--------------------+------+
|Agilent Technolog...|         Health Care|     A|
|American Airlines...|         Industrials|   AAL|
|  Advance Auto Parts|Consumer Discreti...|   AAP|
|          Apple Inc.|Information Techn...|  AAPL|
|         AbbVie Inc.|         Health Care|  ABBV|
|AmerisourceBergen...|         Health Care|   ABC|
|         ABIOMED Inc|         Health Care|  ABMD|
| Abbott Laboratories|         Health Care|   ABT|
|       Accenture plc|Information Techn...|   ACN|
|          Adobe Inc.|Information Techn...|  ADBE|
+--------------------+--------------------+------+
only showing top 10 rows



In [7]:
stockspark.show(10)


+-----+----------+---------+-----+-----+----+-----+---------+
|close|      date|dividends| high|  low|name| open|   volume|
+-----+----------+---------+-----+-----+----+-----+---------+
| 85.4|01/02/2020|      0.0| 85.8|84.65|   A|85.35|1410500.0|
|84.03|01/03/2020|      0.0|84.78|83.96|   A|84.13|1118300.0|
|84.28|01/06/2020|      0.0|84.28|83.06|   A|83.46|1993200.0|
|84.53|01/07/2020|      0.0|84.71| 83.4|   A|83.42|1684700.0|
|85.37|01/08/2020|      0.0|85.91|84.65|   A|85.41|1847600.0|
|86.71|01/09/2020|      0.0|87.14|85.62|   A| 85.9|1912700.0|
|87.03|01/10/2020|      0.0|87.67|86.76|   A|87.16|1417000.0|
| 86.9|01/13/2020|      0.0|87.75|86.18|   A|87.25|1630200.0|
|87.43|01/14/2020|      0.0|87.64|86.14|   A|86.71|1675200.0|
|88.05|01/15/2020|      0.0|88.54|86.99|   A|87.07|1630400.0|
+-----+----------+---------+-----+-----+----+-----+---------+
only showing top 10 rows



In [6]:
sectorjoin=stockspark.join(companyspark,stockspark.name==companyspark.Symbol).select(stockspark.date,companyspark.Symbol,stockspark.open,stockspark.close,stockspark.high,stockspark.low,stockspark.volume,stockspark.dividends,companyspark.Sector)

In [7]:
sectorjoin.show(5)


+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|      date|Symbol| open|close| high|  low|   volume|dividends|     Sector|
+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|01/02/2020|     A|85.35| 85.4| 85.8|84.65|1410500.0|      0.0|Health Care|
|01/03/2020|     A|84.13|84.03|84.78|83.96|1118300.0|      0.0|Health Care|
|01/06/2020|     A|83.46|84.28|84.28|83.06|1993200.0|      0.0|Health Care|
|01/07/2020|     A|83.42|84.53|84.71| 83.4|1684700.0|      0.0|Health Care|
|01/08/2020|     A|85.41|85.37|85.91|84.65|1847600.0|      0.0|Health Care|
+----------+------+-----+-----+-----+-----+---------+---------+-----------+
only showing top 5 rows



In [8]:
sectorjoin[['Sector']].distinct().show()


+--------------------+
|              Sector|
+--------------------+
|         Health Care|
|Communication Ser...|
|              Energy|
|Information Techn...|
|         Real Estate|
|           Materials|
|Consumer Discreti...|
|           Utilities|
|    Consumer Staples|
|         Industrials|
|          Financials|
+--------------------+



In [21]:
#sectorjoin.groupBy(['Symbol','Sector','date']).agg({'close':'max'}).show()
sectorjoin.groupBy(['Symbol','Sector','date']).agg(fc.max('close').alias('2020HIGH')).show()


+------+--------------------+----------+--------+
|Symbol|              Sector|      date|2020HIGH|
+------+--------------------+----------+--------+
|     A|         Health Care|03/25/2020|   68.56|
|   AAL|         Industrials|03/04/2020|   18.53|
|  ABBV|         Health Care|07/15/2020|   99.13|
|   ABC|         Health Care|06/19/2020|  101.73|
|  ABMD|         Health Care|01/17/2020|  181.28|
|  ABMD|         Health Care|05/04/2020|   181.0|
|   ACN|Information Techn...|04/06/2020|  164.06|
|   ACN|Information Techn...|08/18/2020|  233.32|
|  ADBE|Information Techn...|02/26/2020|  351.34|
|   ADI|Information Techn...|03/19/2020|   89.37|
|   ADM|    Consumer Staples|05/20/2020|   35.06|
|   ADP|Information Techn...|02/12/2020|  174.74|
|   AEE|           Utilities|01/21/2020|   78.54|
|   AFL|          Financials|06/02/2020|   36.96|
|   AFL|          Financials|07/08/2020|   35.14|
|   AIG|          Financials|03/11/2020|   31.35|
|   AIG|          Financials|07/28/2020|   30.61|


In [26]:
new1=sectorjoin[(sectorjoin.Sector=='Health Care')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('_2020HIGH'))


In [48]:
new1.show()

+------+-----------+---------+
|Symbol|     Sector|_2020HIGH|
+------+-----------+---------+
|  AMGN|Health Care|   259.21|
|  INCY|Health Care|   109.69|
|  REGN|Health Care|   658.21|
|   MCK|Health Care|   183.44|
|  ILMN|Health Care|   400.74|
|   CAH|Health Care|    58.65|
|  PRGO|Health Care|    60.07|
|  ALXN|Health Care|   127.62|
|   BSX|Health Care|    45.71|
|   STE|Health Care|   191.82|
|   MDT|Health Care|   119.08|
|   HUM|Health Care|   452.01|
|  ABBV|Health Care|    99.48|
|   SYK|Health Care|   234.35|
|   WST|Health Care|   301.49|
|    LH|Health Care|   214.42|
|   MYL|Health Care|    22.85|
|  HSIC|Health Care|    73.71|
|    CI|Health Care|   222.51|
|   ZTS|Health Care|   174.35|
+------+-----------+---------+
only showing top 20 rows



In [42]:
new2=new1.join(sectorjoin,(new1.Symbol==sectorjoin.Symbol)).select(new1.Symbol,new1.Sector,sectorjoin.close,new1._2020HIGH,sectorjoin.date)


In [49]:
new2.show(5)

+------+-----------+------+---------+----------+
|Symbol|     Sector| close|_2020HIGH|      date|
+------+-----------+------+---------+----------+
|  ALXN|Health Care|107.84|   127.62|01/02/2020|
|  ALXN|Health Care|106.41|   127.62|01/03/2020|
|  ALXN|Health Care|106.58|   127.62|01/06/2020|
|  ALXN|Health Care|106.85|   127.62|01/07/2020|
|  ALXN|Health Care|108.58|   127.62|01/08/2020|
+------+-----------+------+---------+----------+
only showing top 5 rows

